In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from collections import namedtuple
import timeit

import tensorflow as tf
from tensorflow import keras

In [2]:
EPOCHS = 1
TRAIN_REPETITIONS = 10
EVALUATION_REPETITIONS = 10
PREDICTION_REPETITIONS = 10
Examples = namedtuple('Examples', 'images labels')
Data = namedtuple('Data', 'train test')

In [3]:
def main():
    data = get_data()
    model = build_model()

    train_samples = len(data.train.images)
    test_samples = len(data.test.images)

    g = globals()
    g['data'] = data
    g['model'] = model

    print('== TRAINING ==')
    train_time = timeit.timeit(
        'train_model(model, data)',
        globals=g,
        number=TRAIN_REPETITIONS)
    train_time_per_epoch = train_time / (EPOCHS * TRAIN_REPETITIONS)
    train_time_per_sample = train_time_per_epoch / train_samples
    print(f'Time per epoch estimate: {train_time_per_epoch}')
    print(f'Time per sample estimate: {train_time_per_sample}')

    print('== EVALUATING ==')
    evaluate_time = timeit.timeit(
        'evaluate_model(model, data)',
        globals=g,
        number=EVALUATION_REPETITIONS)
    evaluation_time_per_epoch = evaluate_time / EVALUATION_REPETITIONS
    evaluation_time_per_sample = evaluation_time_per_epoch / test_samples
    print(f'Time per epoch estimate: {evaluation_time_per_epoch}')
    print(f'Time per sample estimate: {evaluation_time_per_sample}')

    print('== PREDICTING ==')
    prediction_time = timeit.timeit(
        'make_predictions(model, data)',
        globals=g,
        number=PREDICTION_REPETITIONS)
    prediction_time_per_epoch = prediction_time / PREDICTION_REPETITIONS
    prediction_time_per_sample = prediction_time_per_epoch / test_samples
    print(f'Time per epoch estimate: {prediction_time_per_epoch}')
    print(f'Time per sample estimate: {prediction_time_per_sample}')


In [4]:
def build_model():
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(10)
    ])
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])
    return model

In [5]:
def get_data():
    fashion_mnist = keras.datasets.fashion_mnist
    train, test = fashion_mnist.load_data()
    train_images, train_labels = train
    test_images, test_labels = test
    train_images = train_images / 255.0
    test_images = test_images / 255.0
    return Data(
        train=Examples(images=train_images, labels=train_labels),
        test=Examples(images=test_images, labels=test_labels))


In [6]:
def train_model(model, data):
    model.fit(
        data.train.images,
        data.train.labels,
        epochs=EPOCHS,
        verbose=0)

In [7]:
def evaluate_model(model, data):
    model.evaluate(
        data.test.images,
        data.test.labels,
        verbose=0)

In [8]:
def make_predictions(model, data):
    probability_model = tf.keras.Sequential([
        model,
        tf.keras.layers.Softmax()])
    probability_model.predict(data.test.images)


In [9]:
if __name__ == '__main__':
    main()

4423680/4422102 [==============================] - 0s 0us/step
== TRAINING ==
Time per epoch estimate: 7.550293586399993
Time per sample estimate: 0.00012583822643999989
== EVALUATING ==
Time per epoch estimate: 0.7031205795000005
Time per sample estimate: 7.031205795000005e-05
== PREDICTING ==
Time per epoch estimate: 0.6618956165999862
Time per sample estimate: 6.618956165999862e-05
